In [1]:
import os
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from model_settings import ms
from convsklearn import asian_trainer,barrier_trainer

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

root = Path().resolve().parent.parent

In [2]:
trainer = barrier_trainer
models_dir = os.path.join(root,ms.trained_models)
models = [f for f in os.listdir(models_dir) if f.find('ipynb')==-1]
for i,m in enumerate(models):
    print(f"{i}     {m}")

0     2024_10_31 203810934918 tuned bloomberg barriers
1     2024_10_31 204451887463 tuned bloomberg asians
2     2024_11_01 094924306208 cboe asians


In [3]:
model = models[0]
model_dir = os.path.join(models_dir,model)
model_files = [f for f in os.listdir(model_dir) if f.find('ipynb')==-1 and f.find('.html')==-1]
for i,m in enumerate(model_files):
    print(f"{i}     {m}")

0     2024_10_31 203810934918 tuned bloomberg barriers test_data.csv
1     2024_10_31 203810934918 tuned bloomberg barriers.pkl
2     2024_10_31 203810934918 tuned bloomberg barriers train_data.csv


In [4]:
trainer.feature_set

['spot_price',
 'strike_price',
 'days_to_maturity',
 'risk_free_rate',
 'dividend_rate',
 'kappa',
 'theta',
 'rho',
 'eta',
 'v0',
 'barrier',
 'barrier_type_name',
 'w']

In [5]:
train_data = pd.read_csv(os.path.join(model_dir,[f for f in model_files if f.find('train')!=-1][0])).iloc[:,1:].copy()
test_data = pd.read_csv(os.path.join(model_dir,[f for f in model_files if f. find('test')!=-1][0])).iloc[:,1:].copy()
arrs = trainer.get_train_test_arrays(train_data,test_data)
train_X = trainer.get_X(train_data)
train_y = trainer.get_y(train_data)
test_X = trainer.get_X(test_data)
test_y = trainer.get_y(test_data)


n = len(trainer.feature_set)

numerical_scaler = StandardScaler()

feature_transformers = [
        ('numerical', numerical_scaler, trainer.numerical_features),
        ('categorical', OneHotEncoder(sparse_output=False), trainer.categorical_features)
    ]

In [6]:
preprocessor = ColumnTransformer(
    transformers=feature_transformers
)

dnn_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('dnn', MLPRegressor(**trainer.dnn_params))
])

scaled_dnn = TransformedTargetRegressor(
    regressor=dnn_pipeline,
    transformer=numerical_scaler
)

In [ ]:
param_grid = {
    # 'regressor__dnn__activation': ['identity', 'logistic', 'tanh', 'relu'],
    # 'regressor__dnn__solver': ['lbfgs', 'sgd', 'adam'],
    'regressor__dnn__alpha': [0.0001, 0.001, 0.01, 0.1],
    # 'regressor__dnn__batch_size': ['auto', 32, 64, 128],
    'regressor__dnn__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'regressor__dnn__learning_rate_init': [0.001, 0.01, 0.1],
    # 'regressor__dnn__power_t': [0.5, 0.25],
    # 'regressor__dnn__max_iter': [200, 500, 1000],
    # 'regressor__dnn__shuffle': [True, False],
    # 'regressor__dnn__tol': [1e-4, 1e-3, 1e-2],
    'regressor__dnn__warm_start': [False, True],
    # 'regressor__dnn__momentum': [0.9, 0.8, 0.7],
    # 'regressor__dnn__nesterovs_momentum': [True, False],
    'regressor__dnn__early_stopping': [False, True],
    # 'regressor__dnn__validation_fraction': [0.1, 0.15, 0.2],
    # 'regressor__dnn__beta_1': [0.9, 0.85],
    # 'regressor__dnn__beta_2': [0.999, 0.99],
    # 'regressor__dnn__epsilon': [1e-8, 1e-6],
    # 'regressor__dnn__n_iter_no_change': [10, 20],
    # 'regressor__dnn__max_fun': [15000, 20000]
}

grid_search = GridSearchCV(
    estimator=scaled_dnn,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=max(1,os.cpu_count()//4),
    verbose=1,
    error_score='raise'
)

grid_search.fit(train_X, train_y)

print("Best Parameters:", grid_search.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
